In [1]:
import os, requests, json
import pandas as pd
import geemap
import ee
#ee.Authenticate()

In [2]:
ee.Initialize()

In [3]:
# Get ULU polygons

lu1 = ee.ImageCollection('projects/wri-datalab/cities/urban_land_use/V1')
lu2 = ee.ImageCollection('projects/wri-datalab/urban_land_use/V2')
lu = lu1.merge(lu2).select('lulc')

In [61]:
# Get WorldPop data

pop = ee.ImageCollection('WorldPop/GP/100m/pop').filter(ee.Filter.equals('year', 2010))

In [4]:
with open('country_list.csv', 'r') as ifile:
    countries = [[j.strip() for j in line.split(',')] for line in ifile.readlines()]

country_boundaries = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0")

In [5]:
REDUCER_SCALES = [100, 200, 500, 1000, 1500, 2000, 2500, 5000]

In [75]:
all_fc = ee.FeatureCollection([])
for country_code, country_name, FAO_country_name in countries:
    if country_code == 'ETH':
        all_cities = lu.filter(ee.Filter.equals("Country__Name", country_name))
        country_feat = country_boundaries.filter(ee.Filter.equals('ADM0_NAME', FAO_country_name))
        bysize = all_cities.sort("Pop_2010", False)
        thelist = bysize.select('lulc').toList(1000000)
        metro_names = []
        already_found = []
        for i in [j['properties']['City__Name'] for j in thelist.getInfo()]:
            if not i in already_found:
                metro_names.append(i)
                already_found.append(i)
                if len(i.split('__')) > 1:
                    for j in i.split('__'):
                        already_found.append(j.strip()
                                             
                                             
    ee_metronames = ee.List(metro_names)
    
    country_pop = pop.filter(ee.Filter.equals('country', country_code)).select('population').first()

    def get_fc(metro_name):
        lu_local = lu.filter(ee.Filter.equals("Country__Name", country_name)).filter(ee.Filter.equals("City__Name", metro_name)).mosaic()
        lu_area = ee.Image.pixelArea().addBands(lu_local.select("lulc").reduce(ee.Reducer.firstNonNull()))

        areas = lu_area.reduceRegion(
                reducer= ee.Reducer.sum().group(
                groupField= 1,
                groupName= 'lulc',
            ),
            geometry= country_feat.geometry(),
            scale= 50,
            maxPixels= 1e12
        )
        
        total_pop_raster = lu_local.select("lulc").gt(0).multiply(country_pop)
        total_pop_sum = total_pop_raster.reduceRegion(ee.Reducer.sum(), country_feat.geometry(), 100, maxPixels= 1e12)

        
        return ee.Feature(country_feat.geometry(), {
            "country_name": country_name,
            "metro_name": metro_name,
            "population_2010": ee.Number.float(ee.Dictionary(total_pop_sum).get("lulc")),
            "openspace_area": ee.Number.float((ee.List(ee.Dictionary(areas).get("groups")).map(lambda x: ee.Number.float(ee.Dictionary(x).get("sum")).multiply(ee.Number.int(ee.Number.int(ee.Dictionary(x).get('lulc')).eq(0))))).reduce(ee.Reducer.sum())),
            "nonresidential_area": ee.Number.float((ee.List(ee.Dictionary(areas).get("groups")).map(lambda x: ee.Number.float(ee.Dictionary(x).get("sum")).multiply(ee.Number.int(ee.Number.int(ee.Dictionary(x).get('lulc')).eq(1))))).reduce(ee.Reducer.sum())),
            "atomisticsubdiv_area": ee.Number.float((ee.List(ee.Dictionary(areas).get("groups")).map(lambda x: ee.Number.float(ee.Dictionary(x).get("sum")).multiply(ee.Number.int(ee.Number.int(ee.Dictionary(x).get('lulc')).eq(2))))).reduce(ee.Reducer.sum())),
            "informalsubdiv_area": ee.Number.float((ee.List(ee.Dictionary(areas).get("groups")).map(lambda x: ee.Number.float(ee.Dictionary(x).get("sum")).multiply(ee.Number.int(ee.Number.int(ee.Dictionary(x).get('lulc')).eq(3))))).reduce(ee.Reducer.sum())),
            "formalsubdiv_area": ee.Number.float((ee.List(ee.Dictionary(areas).get("groups")).map(lambda x: ee.Number.float(ee.Dictionary(x).get("sum")).multiply(ee.Number.int(ee.Number.int(ee.Dictionary(x).get('lulc')).eq(4))))).reduce(ee.Reducer.sum())),
            "housingproject_area": ee.Number.float((ee.List(ee.Dictionary(areas).get("groups")).map(lambda x: ee.Number.float(ee.Dictionary(x).get("sum")).multiply(ee.Number.int(ee.Number.int(ee.Dictionary(x).get('lulc')).eq(5))))).reduce(ee.Reducer.sum())),
            "road_area": ee.Number.float((ee.List(ee.Dictionary(areas).get("groups")).map(lambda x: ee.Number.float(ee.Dictionary(x).get("sum")).multiply(ee.Number.int(ee.Number.int(ee.Dictionary(x).get('lulc')).eq(6))))).reduce(ee.Reducer.sum()))
            })
    
    results = ee_metronames.map(get_fc)
    results_fc = ee.FeatureCollection(results)
    
    task = ee.batch.Export.table.toAsset(results_fc, "informal_builtup_{}".format(country_name.replace(' ','_')), "projects/ee-tedwongwri/assets/informal_builtup_{}".format(country_name.replace(' ','_')))
    task.start()
    all_fc = all_fc.merge(results_fc)
task = ee.batch.Export.table.toAsset(all_fc, "informal_builtup_allcountries", "projects/ee-tedwongwri/assets/informal_builtup_allcountries")
task.start()
    

'                                            \n    ee_metronames = ee.List(metro_names)\n    \n    country_pop = pop.filter(ee.Filter.equals(\'country\', country_code)).select(\'population\').first()\n\n    def get_fc(metro_name):\n        lu_local = lu.filter(ee.Filter.equals("Country__Name", country_name)).filter(ee.Filter.equals("City__Name", metro_name)).mosaic()\n        lu_area = ee.Image.pixelArea().addBands(lu_local.select("lulc").reduce(ee.Reducer.firstNonNull()))\n\n        areas = lu_area.reduceRegion(\n                reducer= ee.Reducer.sum().group(\n                groupField= 1,\n                groupName= \'lulc\',\n            ),\n            geometry= country_feat.geometry(),\n            scale= 50,\n            maxPixels= 1e12\n        )\n        \n        total_pop_raster = lu_local.select("lulc").gt(0).multiply(country_pop)\n        total_pop_sum = total_pop_raster.reduceRegion(ee.Reducer.sum(), country_feat.geometry(), 100, maxPixels= 1e12)\n\n        \n        re